# Isaac Sim App Launcher

1. Click "Start App Launcher" button below
2. Wait for the init process
3. Choose isaac-sim examples

<button data-commandlinker-command="notebook:run-all-cells" class="jupyter-button" style="color: #fff;background-color: #1976d2;">Start App Launcher</button>
<button data-commandlinker-command="terminal:create-new" data-commandlinker-args='{"cwd": "dev-tools"}' class="jupyter-button">Open a Terminal</button>
<!-- <button data-commandlinker-command="notebook:interrupt-kernel" class="jupyter-button mod-danger">Stop All</button> -->

In [ ]:
import os
import signal
import html
import time
from utils import *
import subprocess
import threading
import ipywidgets as widgets
import textwrap
from pathlib import Path
from IPython.display import display, clear_output, Markdown, Javascript

# Define environment variables
os.environ["ROS_DOMAIN_ID"] = "0"
os.environ["RMW_IMPLEMENTATION"] = "rmw_fastrtps_cpp"
os.environ["ROS_AUTOMATIC_DISCOVERY_RANGE"] = "LOCALHOST"
os.environ["DEV_TOOLS_PATH"] = "/mnt/dev-tools"
os.environ["ISAACSIM_VERSION"] = "5.0"
os.environ["ISAACSIM_PATH"] = f"{os.environ['DEV_TOOLS_PATH']}/isaac-sim-{os.environ['ISAACSIM_VERSION']}"
os.environ["ISAACSIM_PYTHON_EXE"] = f"{os.environ['ISAACSIM_PATH']}/python.sh"
# unset virtualGL
# os.environ.pop("LD_PRELOAD", None)

APP_LIST = [
    {
        "name": "Main App",
        "path": "isaac-sim.sh",
        "command": textwrap.dedent("""
            $ISAACSIM_PATH/isaac-sim.sh
        """)
    },
    {
        "name": "Main App Streaming",
        "path": "isaac-sim.streaming.sh",
        "command": textwrap.dedent("""
            $ISAACSIM_PATH/isaac-sim.streaming.sh &
            $DEV_TOOLS_PATH/streaming-client/isaacsim-webrtc-streaming-client --no-sandbox
        """)
    },
    {
        "name": "Streaming Client",
        "path": "",
        "command": textwrap.dedent("""
            $DEV_TOOLS_PATH/streaming-client/isaacsim-webrtc-streaming-client --no-sandbox
        """)
    },
    {
        "name": "Franka Panda (ROS2)",
        "path": f"{os.getcwd()}/moveit.py"
    },
    {
        "name": "Nova Carter (ROS2)",
        "path": f"{os.getcwd()}/carter_stereo.py",
        "command": textwrap.dedent(f"""
            $ISAACSIM_PYTHON_EXE {os.getcwd()}/carter_stereo.py &
            source /mnt/dev-tools/ros2_ws/install/setup.bash
            rviz2 -d {os.getcwd()}/carter_stereo.rviz &
            ros2 run rqt_robot_steering rqt_robot_steering
        """)
    },
    {
        "name": "Multiple Robot",
        "path": f"{os.getcwd()}/carter_multiple_robot_navigation.py",
        "command": textwrap.dedent(f"""
            $ISAACSIM_PYTHON_EXE {os.getcwd()}/carter_multiple_robot_navigation.py &
            source $DEV_TOOLS_PATH/ros2_ws/install/setup.bash
            ros2 run rqt_robot_steering rqt_robot_steering
        """)
    },
    {
        "name": "RL Policy",
        "path": f"{os.getcwd()}/policy.py"
    },
    {
        "name": "UR10 conveyor",
        "path": f"{os.getcwd()}/demo_ur10_conveyor_main.py"
    }
]

# Add environment setup
for i in APP_LIST:
    if 'command' not in i:
        i['command'] = f"$ISAACSIM_PYTHON_EXE {i['path']}"
    i['command'] = ISAACSIM_ENV + i['command']

# List all offical standalone_examples
examples_dir = Path(f"{os.environ['ISAACSIM_PATH']}/standalone_examples")
examples_list = [
    {
        "name": f.stem,
        "path": str(f),
        "command": ISAACSIM_ENV +  f"$ISAACSIM_PYTHON_EXE {str(f)}"
    }
    
    for f in examples_dir.rglob("*.py")
    if ".ipynb_checkpoints" not in f.parts and f.name != "__init__.py"
]

# Run one process at a time
current_process = None
lock = threading.Lock()

def run_script(script, label):
    global current_process
    with lock:
        # if current_process is not None and current_process.poll() is None:
        #     try:
        #         # os.killpg(os.getpgid(current_process.pid), signal.SIGTERM
        #         current_process.terminate()
        #     except Exception as e:
        #         print("Kill previous process failed:", e)
        #     current_process.wait()
        current_process = subprocess.Popen(
            ["bash", "-c", script],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            # preexec_fn=os.setsid
        )
    
    output_area.value = ""
    output_label.value = f'Running "{label}"'
    for line in current_process.stdout:
        output_area.value  = line + output_area.value

    current_process.wait()

# Generate launcher buttons
def make_button(data, script_type="bash", button_style="primary", min_width='50px', show_source=False):
    label = data['name']
    script = data['command']
    button = widgets.Button(description=label,
                            button_style=button_style,
                            tooltip=script,
                            layout=widgets.Layout(min_width=min_width))
    def on_click(b):
        match script_type:
            case _:
                thread = threading.Thread(target=run_script, args=(script, label))
                thread.start()

    button.on_click(on_click)

    if show_source:
        file_path = os.path.abspath(os.path.join(os.environ["ISAACSIM_PATH"], data['path']))
        try:
            JUPYTERHUB_USER = os.environ['JUPYTERHUB_USER']
        except KeyError:
            JUPYTERHUB_USER = None
        url_prefix = f"/user/{JUPYTERHUB_USER}" if JUPYTERHUB_USER is not None else ''
        url_param = f'folder={os.path.dirname(file_path)}&payload=[["openFile","vscode-remote://{file_path}"]]'
        url_param = html.escape(url_param)
        src_btn = widgets.HTML(
            value=f'<a href="{url_prefix}/vscode/?{url_param}" title="{file_path}" class="jupyter-button" style="width:100%;padding:0;" target="_blank">Source Code</a>',
        )
        
        return widgets.VBox([button, src_btn])
    else:
        return button

# Main examples UI
app_btns = widgets.VBox()

# Other tools UI
other_tools_btns = widgets.HBox([
    make_button({"command": RVIZ_CMD,  "name": "Rviz"}),
    make_button({"command": MULTIVERSE_CMD,  "name": "MULTIVERSE"}),
    make_button({"command": UNREAL_DEMO_CMD, "name": "UE_kitchen"}),
    make_button({"command": APARTMENT_CMD, "name": "IAI_apartment"}),
    make_button({"command": BLENDER_CMD, "name": "Blender"}),
])

# Stop button
control_ui = widgets.HBox([
    widgets.HTML(
        value=f'<button data-commandlinker-command="notebook:interrupt-kernel" style="min-width:120px;" class="jupyter-button mod-danger">Stop All</button>'
    ),
    # make_button({"command": "echo 'App Killed!'", "name": "Stop All"}, button_style="danger"),
    make_button({"command": CACHE_EXTRACT_CMD, "name": "Extract Cache"}, button_style="info"),
    make_button({"command": CACHE_ARCHIVE_CMD, "name": "Archive Cache"}, button_style="info")
])

# Process Output area
output_label = widgets.Label(value="")
output_area = widgets.Textarea(
    rows=15,
    layout={"width": "95%"}
)

# Full Isaac Sim example UI
example_btns = [make_button(i, show_source=True) for i in sorted(examples_list, key=lambda x: x['name'])]
n_cols = 4
rows = []
for i in range(0, len(example_btns), n_cols):
    row = widgets.HBox(example_btns[i:i+n_cols])
    rows.append(row)
examples_list_ui = widgets.VBox(rows)

ui = widgets.VBox([
    output_label,
    output_area
])

# Open remote desktop
display_desktop()

# Show UI
display(ui)

# Extract default cache
# import time
if not os.path.isdir(f"{os.environ['HOME']}/.nv"):
    # start_time = time.time()
    run_script(CACHE_EXTRACT_CMD, "Extract Cache")
    # print(time.time() - start_time)

run_script("echo 'Launcher Ready!'", "Nothing")

# Show example buttons after cache ready
ui.children = [
    widgets.Label(value=f"Isaac Sim {os.environ['ISAACSIM_VERSION']}:"),
    widgets.HBox([make_button(i) for i in APP_LIST[:3]]),
    widgets.Label(value="Isaac sim Python Examples:"),
    widgets.HBox([make_button(i, show_source=True) for i in APP_LIST[3:]]),    
    widgets.Label(value="Other Tools:"),
    other_tools_btns,
    output_label,
    control_ui,
    output_area,
    widgets.Label(value="Full Isaac Sim Examples:"),
    examples_list_ui
]